In [22]:
from sqlalchemy import create_engine
import psycopg2
import arcpy
import pandas as pd
import random
import numpy as np
import json

In [23]:
temp_csv_path = r'C:\Users\tjjoh\OneDrive\Desktop\GIS 5572\Lab3\temperature.csv'

stem_path = r'C:\Users\tjjoh\Documents\GIS5572'
gdb_name = r'Lab3GDB'
gdb_path = stem_path + r'/' + gdb_name + '.gdb'

In [24]:
#create database connection
connection_string = f'postgresql://<user>:<password>@34.133.43.30:5432/lab2'
engine = create_engine(connection_string)

#retrieve the data from PostGIS
table_name = "mn_temperature"
query = f'SELECT *, ST_AsText(geometry) AS WKT_geom FROM {table_name};'

df = pd.read_sql(query, engine) #read data with pandas
df.to_csv(temp_csv_path, index = False) #save as csv

In [25]:
#create a new file GDB if one doesn't exist
try:
    arcpy.management.CreateFileGDB(
        out_folder_path = stem_path,
        out_name = gdb_name,
        out_version = 'CURRENT'
    )
except:
    print('GDB already exists.')

In [26]:
#set workspace
arcpy.env.workspace = gdb_path

#create a new feature class
temperature = 'temperature'
spatial_ref = arcpy.SpatialReference(4326)
arcpy.management.CreateFeatureclass(gdb_path, temperature, 'POINT', spatial_reference = spatial_ref)

#add a field for elevation to the feature class
months = ['January', 'February', 'March', 'April', 'May', 'June', 
          'July', 'August', 'September', 'October', 'November', 'December']
for month in months:
    arcpy.management.AddField(temperature, f'{month}', 'DOUBLE')

#iterate through the dataframe to populate ferature class
cursor = arcpy.da.InsertCursor(temperature, ['SHAPE@', 'January', 'February', 'March', 'April', 'May', 'June', 
                                             'July', 'August', 'September', 'October', 'November', 'December'])
for index, row in df.iterrows():
    geometry = arcpy.FromWKT(row['wkt_geom'])  #convert WKT to geometry
    cursor.insertRow([geometry, row['January'], row['February'], row['March'], row['April'], row['May'], row['June'],
                      row['July'], row['August'], row['September'], row['October'], row['November'], row['December']])

del cursor

In [27]:
#clear selection
arcpy.management.SelectLayerByAttribute(temperature, "CLEAR_SELECTION")

#create training and testing datasets
training = 'training'
testing = 'testing'

#count features
total_count = int(arcpy.GetCount_management(temperature)[0])

#calculate 50% sample size
sample_size = max(1, total_count // 2)  # Ensure at least one feature is selected

#get all OBJECTIDs
oids = [row[0] for row in arcpy.da.SearchCursor(temperature, ["OID@"])]

#randomly select 50% of features
random_oids = random.sample(oids, sample_size)

#create selection query
oid_query = f"OBJECTID IN ({','.join(map(str, random_oids))})"

#select and export training data
arcpy.management.SelectLayerByAttribute(temperature, "NEW_SELECTION", oid_query)
arcpy.management.CopyFeatures(temperature, training)

#create opposite selection query
oid_query = f"OBJECTID NOT IN ({','.join(map(str, random_oids))})"

#select and export testing data
arcpy.management.SelectLayerByAttribute(temperature, "NEW_SELECTION", oid_query)
arcpy.management.CopyFeatures(temperature, testing)

<Result 'C:\\Users\\tjjoh\\Documents\\GIS5572/Lab3GDB.gdb\\testing'>

In [28]:
#conduct spline interpolation on training data
spline = arcpy.ddd.Spline(
    in_point_features = training,
    z_field = 'December',
    out_raster = gdb_path + r'/Spline',
)

In [29]:
#conduct IDW interpolation on training data
idw = arcpy.ddd.Idw(
    in_point_features = training,
    z_field = 'December',
    out_raster = gdb_path + r'/IDW',
    power = 2,
)

In [30]:
#conduct Ordinary Kriging on training data
okriging = arcpy.ddd.Kriging(
    in_point_features = training,
    z_field = 'December',
    out_surface_raster = gdb_path + r'/OKriging',
)

In [31]:
#extract the values of the interpolated rasters to the testing data points
arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features = testing,
    in_rasters = 'Spline Spline;IDW IDW;OKriging OKriging',
)

<geoprocessing server result object object at 0x000002121ACACFC0>

In [32]:
with arcpy.da.UpdateCursor(testing, ['IDW']) as cursor:
    for row in cursor:
        if row[0] is None:  # Check for null value
            cursor.deleteRow()  # Remove feature

In [33]:
methods = ['Spline', 'IDW', 'OKriging']

#calculate error for each testing point for each method
for method in methods:
    arcpy.management.CalculateField(
        in_table = testing,
        field = f'error_{method}',
        expression = f'!{method}! - !December!',
        expression_type = 'PYTHON3',
        field_type = 'FLOAT',
    )

In [34]:
#extract errors for each method as a list
error_spline = [row[0] for row in arcpy.da.SearchCursor(testing, ['error_Spline'])]
error_idw = [row[0] for row in arcpy.da.SearchCursor(testing, ['error_IDW'])]
error_okrig = [row[0] for row in arcpy.da.SearchCursor(testing, ['error_OKriging'])]

In [35]:
#calculate rmse, mae, and maxae for each method
rmse = []
mae = []
maxae = []
for error in [error_spline, error_idw, error_okrig]:
    absolute_error = [abs(x) for x in error]
    squared_error = [x ** 2 for x in error]
    
    rmse.append(np.sqrt(np.mean(squared_error)))
    mae.append(np.mean(absolute_error))
    maxae.append(np.max(absolute_error))

In [36]:
#create a pandas dataframe for accuracy assessment
accuracy_assessment = pd.DataFrame({'Interpolation Method': methods,
                                   'Root Mean Squared Error': rmse,
                                   'Mean Absolute Error': mae,
                                   'Maximum Absolute Error': maxae
                                   })

In [37]:
accuracy_assessment

,Interpolation Method,Root Mean Squared Error,Mean Absolute Error,Maximum Absolute Error
0,Spline,2.176991,1.641427,8.602092
1,IDW,1.179037,0.924770,3.671749
2,OKriging,1.157111,0.908470,3.447047


In [38]:
# Push the DataFrame to PostGIS
table_name = 'temperature_accuracy'
accuracy_assessment.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"DataFrame successfully pushed to the PostGIS table '{table_name}'.")

DataFrame successfully pushed to the PostGIS table 'temperature_accuracy'.


In [39]:
testing_geojson = stem_path + r'\Lab3\testing_points.geojson'

arcpy.conversion.FeaturesToJSON(
    in_features = testing,
    out_json_file = testing_geojson,
    geoJSON = 'GEOJSON'
)

<Result 'C:\\Users\\tjjoh\\Documents\\GIS5572\\Lab3\\testing_points.geojson'>

In [40]:
with open(testing_geojson, "r") as f:
    geojson_data = json.load(f)

In [41]:
geojson_data['features'][0]

{'type': 'Feature', 'id': 1, 'geometry': {'type': 'Point', 'coordinates': [-95.98159999999996, 48.300500000000056]}, 'properties': {'OBJECTID': 1, 'January': 5.9, 'February': 11.7, 'March': 25.7, 'April': 42.9, 'May': 56.3, 'June': 65.3, 'July': 69.9, 'August': 67.9, 'September': 58, 'October': 44, 'November': 26.6, 'December': 10.8, 'Spline': 10.690032, 'IDW': 10.7599697, 'OKriging': 10.5148058, 'error_Spline': -0.109967992, 'error_IDW': -0.0400302894, 'error_OKriging': -0.285194218}}

In [42]:
table_name = 'temperature_testing'
conn = engine.connect()
conn.execute(f"DROP TABLE IF EXISTS {table_name};")
conn.execute(f"""
    CREATE TABLE {table_name} (
        id SERIAL PRIMARY KEY,
        geom GEOMETRY,
        attributes JSONB
    );
""")

i = 0
for feature in geojson_data["features"]:
    if i < 2000:
        geom = json.dumps(feature["geometry"])  # Convert geometry to JSON
        properties = feature["properties"]  # Extract attributes

        insert_query = f"""
            INSERT INTO {table_name} (geom, attributes)
            VALUES (ST_GeomFromGeoJSON('{geom}'), '{json.dumps(properties)}')
        """
        conn.execute(insert_query)
        
        i += 1
        
    else:
        break